In [1]:
import os

with open(".env") as f:
    for line in f:
        # Ignore comments and empty lines
        if line.strip() and not line.startswith("#"):
            key, value = line.strip().split("=", 1)
            os.environ[key] = value

In [2]:
from llama_index.multi_modal_llms.gemini import GeminiMultiModal

from llama_index.core.multi_modal_llms.generic_utils import load_image_urls

image_urls = [
    "https://storage.googleapis.com/generativeai-downloads/data/scene.jpg",
    # Add yours here!
]

image_documents = load_image_urls(image_urls)

gemini_pro = GeminiMultiModal(model_name="models/gemini-pro-vision")

d:\gemini-multi-modal\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to d:\gemini-multi-
[nltk_data]     modal\venv\lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to d:\gemini-multi-
[nltk_data]     modal\venv\lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Unzipping tokenizers\punkt.zip.
